In [ ]:
import xgboost as xgb
from sklearn.datasets import load_svmlight_file

#  This script demonstrate how to do ranking with XGBRanker
x_train, y_train = load_svmlight_file("mq2008.train")
x_valid, y_valid = load_svmlight_file("mq2008.vali")
x_test, y_test = load_svmlight_file("mq2008.test")

display(x_train)

group_train = []
with open("mq2008.train.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_train.append(int(line.split("\n")[0]))

group_valid = []
with open("mq2008.vali.group", "r") as f:
    data = f.readlines()
    for line in data:
#         print(line)
        group_valid.append(int(line.split("\n")[0]))

params = {'objective': 'rank:ndcg', 'learning_rate': 0.1,
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 6, 'n_estimators': 4}

# model = xgb.sklearn.XGBRanker(**params)
# model.fit(x_train, y_train, group_train, 
#           verbose=True,
#           eval_set=[(x_valid, y_valid)], 
#           eval_group=[group_valid])

model2 = xgb.sklearn.XGBRanker(**params)
model2.fit(x_train, y_train, group_train, 
          verbose = True)

# pred = model.predict(x_test)

pred2 = model2.predict(x_test)

# print(pred)
print(pred2)

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split

df = pd.read_pickle("../pickles/df_small_clean.pkl")

df = df.drop(columns=['PriceBand'])

x_train, x_test, y_train, y_test = train_test_split(df, df["position"], test_size=0.2, random_state=42)

query_lengths = []
[query_lengths.append(len(group)) for srchid, group in x_train.groupby('srch_id')]

params = {'objective': 'rank:ndcg', 'learning_rate': 0.1,
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 20, 'n_estimators': 4}


groups = df.columns

model = xgb.sklearn.XGBRanker(**params)
model.fit(x_train, y_train, query_lengths, verbose=True)

pred = model.predict(x_test)

In [ ]:
print(pred)
display(y_train)
display(x_train)

x_test["rank"] = pred
print(x_test.columns)
display(x_test)